In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
2.14.0-dev20230730


In [2]:
import os

if not os.path.exists("helper_functions.py"):
  !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
else:
  print("[INFO] 'helper_functions.py' already exists, skipping download.")

# Import series of helper functions for the notebook (we've created/used these in previous notebooks)
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

[INFO] 'helper_functions.py' already exists, skipping download.


We will be using RNN to train textual information

**Basic Structure of RNN**
* Input layer
* text_vectorizer(input)
* embedding(x)
* LSTM -> activation = tanh
* output_Dense -> activation = sigmoid

### Get the text dataset

In [3]:
if not os.path.exists("nlp_getting_started.zip"):
    !wget https://storage.googlqeapis.com/ztm_tf_course/nlp_getting_started.zip
    unzip_data("nlp_getting_started.zip")
else:
  print("[INFO] already exists, skipping download.")

[INFO] already exists, skipping download.


### Visualizing a text dataset

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [5]:
train_df.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [6]:
#Shuffle to train data
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head(10)

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
5559,7934,rainstorm,NaN,@Calum5SOS you look like you got caught in a r...,0
1765,2538,collision,NaN,my favorite lady came to our volunteer meeting...,1
1817,2611,crashed,NaN,@brianroemmele UX fail of EMV - people want to...,1
6810,9756,tragedy,"Los Angeles, CA",Can't find my ariana grande shirt this is a f...,0
4398,6254,hijacking,"Athens,Greece",The Murderous Story Of AmericaÛªs First Hijac...,1


In [7]:
# test_data
test_df.head(10)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
5,12,NaN,NaN,We're shaking...It's an earthquake
6,21,NaN,NaN,They'd probably still show more life than Arse...
7,22,NaN,NaN,Hey! How are you?
8,27,NaN,NaN,What a nice hat?
9,29,NaN,NaN,Fuck off!


In [8]:
#How many examples in each class
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [9]:
# visualise some random training examples
import random
random_index = random.randint(0,len(train_df)-5)
print(random_index)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  index,text,target = row
  print(f"Target: {target}, {'(real disaster)' if target > 0 else '(not a real disaster)'}")
  print(f"Text:\n{text}\n")
  print("---")

6363
Target: 1, (real disaster)
Text:
Demolition Means Progress: Flint Michigan and the Fate of the American Metropolis Highsmith https://t.co/ZvoBMDxHGP

---
Target: 1, (real disaster)
Text:
Madhya Pradesh Train Derailment: Village Youth Saved Many Lives

---
Target: 1, (real disaster)
Text:
500 deaths a year from foodborne illness... @frackfreelancs dears... @DECCgovuk @frackfree_eu @tarleton_sophie http://t.co/JSccX8k0jA

---
Target: 0, (not a real disaster)
Text:
namjoon's FANTASTIC IS BOMB BYE OMG

---
Target: 0, (not a real disaster)
Text:
@JulieChen she shouldn't. Being with them is gonna ruin her game and Vanessa is a great player

---


In [10]:
len(train_df_shuffled)

7613

In [11]:
#Splitting the data to create validation -> (train + validation)
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [12]:
#Check length
len(train_sentences),len(train_labels),len(val_sentences),len(val_labels)

(6851, 6851, 762, 762)

In [13]:
# Check few sample
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

1. tokenization -> can get too big as the number of text sample increases
2. Embedding -> richer representation of relationship be between tokens, deeper the realtion between words, more the number/embedding for that token

In [14]:
import tensorflow as tf

# Use the default TextVectorization variables
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                                    standardize="lower_and_strip_punctuation", # how to process text
                                                    split="whitespace", # how to split tokens
                                                    ngrams=None, # create groups of n-words?
                                                    output_mode="int", # how to map tokens to numbers
                                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [15]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = round(sum([len(i.split()) for i in train_sentences])/len(train_sentences)) # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=max_vocab_length,
                                                    output_mode="int",
                                                    output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [18]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
Don't mess with my Daddy I can be a massacre. #BeCarefulHarry      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  63, 2389,   14,   13,    1,    8,   71,   21,    3,  344,    1,
           0,    0,    0,    0]])>

In [20]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [21]:
tf.random.set_seed(42)

embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embedding

In [22]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Toronto going crazy for the blue jays. Can you imagine if the leafs get good? The city might literally explode.

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04486163,  0.02573014, -0.02279862, ...,  0.03354745,
          0.02301952,  0.0467402 ],
        [ 0.00861071,  0.04662026, -0.03669335, ..., -0.04445063,
         -0.0062235 ,  0.0347099 ],
        [-0.01213051,  0.01153048,  0.03086859, ...,  0.04621183,
         -0.00443321, -0.00083055],
        ...,
        [ 0.00433008, -0.02822622, -0.01093443, ...,  0.01338464,
          0.02854312, -0.03799286],
        [ 0.00428232, -0.01403551,  0.03349373, ..., -0.01021522,
          0.04346382,  0.04410707],
        [ 0.03060034, -0.00804458,  0.01401711, ..., -0.02390458,
         -0.0151305 ,  0.01741089]]], dtype=float32)>

In [23]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.04486163,  0.02573014, -0.02279862,  0.03093452,  0.03159383,
       -0.03235666,  0.01509494,  0.00938249,  0.00549501,  0.02309141,
        0.01819383, -0.00858293, -0.02414702,  0.03628812,  0.03950496,
        0.00044477,  0.0281427 ,  0.04461784, -0.04495106, -0.04993987,
        0.04470675, -0.04416573, -0.00659142,  0.00944578, -0.03977437,
        0.03070751, -0.01258197,  0.02826376,  0.02297081, -0.02892262,
       -0.03113362,  0.02961594,  0.0044076 , -0.0210196 ,  0.0055593 ,
       -0.0133971 , -0.03116485,  0.02102199,  0.02906496,  0.01570168,
        0.00256414, -0.02553383, -0.03726934,  0.02964776,  0.00096213,
        0.02992559,  0.01166254,  0.01386534,  0.0266539 , -0.01712762,
        0.04040455, -0.00242213, -0.0348905 ,  0.03203893, -0.00558251,
       -0.02199773, -0.03389901, -0.03932511,  0.01213367,  0.03621433,
       -0.01713515, -0.03027561,  0.01116436, -0.03695067, -0.04738715,
        0.013668

### Modelling a text dataset
More specifically, we'll be building the following:

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model
* Model 3: GRU model
* Model 4: Bidirectional-LSTM model
* Model 5: 1D Convolutional Neural Network
* Model 6: TensorFlow Hub Pretrained Feature Extractor
* Model 7: Same as model 6 with 10% of training data



In [24]:
#model 0
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [26]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [27]:
from helper_functions import calculate_results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

#### Model 1: A simple dense model

In [28]:
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

# Build model with the Functional API
inputs = tf.keras.layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = tf.keras.layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [29]:
# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20230817-221939
Epoch 1/5
215/215 [==============================] - 5s 19ms/step - loss: 0.6090 - accuracy: 0.6944 - val_loss: 0.5354 - val_accuracy: 0.7546
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4406 - accuracy: 0.8192 - val_loss: 0.4690 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3459 - accuracy: 0.8615 - val_loss: 0.4589 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 1s 3ms/step - loss: 0.2844 - accuracy: 0.8923 - val_loss: 0.4640 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 1s 3ms/step - loss: 0.2375 - accuracy: 0.9115 - val_loss: 0.4767 - val_accuracy: 0.7861


In [30]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 1ms/step - loss: 0.4767 - accuracy: 0.7861


[0.47670021653175354, 0.7860892415046692]

In [75]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

24/24 [==============================] - 0s 972us/step


array([[0.3493701 ],
       [0.5704087 ],
       [0.9875553 ],
       [0.14310086],
       [0.02676124],
       [0.8892967 ],
       [0.58423054],
       [0.99571854],
       [0.9833093 ],
       [0.36435312]], dtype=float32)

In [76]:
model_1_preds=tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [77]:
model_1_results=calculate_results(val_labels,model_1_preds)
model_1_results

{'accuracy': 77.03412073490814,
 'precision': 0.7743290047665735,
 'recall': 0.7703412073490814,
 'f1': 0.7671895343790688}

#### Recurrent Neural Network
* RNN's are useful for sequence data
* The premise of a recurrent neural network is to use the representation of previous input to aid the representation of a later input

### Model 2 : LSTM -> Long short term memory

Input(text) -> Toeknize -> Embedding -> Layers(RNN/Dense) -> Output(label probability)

In [31]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)

model_2_embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")


# Create LSTM model
inputs = tf.keras.layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)

print(x.shape)

x = tf.keras.layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.LSTM(64)(x) # return vector for whole sequence

print(x.shape)

outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs)
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_2.summary()

(None, 15, 128)
(None, 64)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 15, 64)            33024     
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                  

In [32]:
# Fit model
model_2_history = model_2.fit(train_sentences, train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "LSTM")])

Saving TensorBoard log files to: model_logs/LSTM/20230817-221948
Epoch 1/5
215/215 [==============================] - 7s 24ms/step - loss: 0.5074 - accuracy: 0.7500 - val_loss: 0.4586 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 2s 7ms/step - loss: 0.3265 - accuracy: 0.8683 - val_loss: 0.5129 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2283 - accuracy: 0.9148 - val_loss: 0.5562 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1687 - accuracy: 0.9426 - val_loss: 0.5890 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1178 - accuracy: 0.9553 - val_loss: 0.8546 - val_accuracy: 0.7559


In [33]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 2ms/step


array([[0.00624254],
       [0.6781507 ],
       [0.99850875],
       [0.12505932],
       [0.00421613],
       [0.9991742 ],
       [0.95853615],
       [0.9992612 ],
       [0.99921453],
       [0.07090925]], dtype=float32)

In [34]:
# Convert preds to laels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [35]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [36]:
model_2_results = calculate_results(val_labels, model_2_preds)
model_2_results

{'accuracy': 75.59055118110236,
 'precision': 0.7565294326194807,
 'recall': 0.7559055118110236,
 'f1': 0.7540879036461907}

#### Model 3: GRU

1. Gated recurrent unit is effective and popular
2. GRU cell has similar features to LSTM cell but has less paramenter

In [37]:
# Build GRU RNN
inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = tf.keras.layers.GRU(64, return_sequences=True)(x)
# x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.GRU(64)(x)
# x = tf.keras.layers.Dense(64,activation="relu")(x)
outputs = tf.keras.layers.Dense(1,activation="sigmoid")(x)

model_3 = tf.keras.Model(inputs,outputs)

model_3.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_3_history = model_3.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences,val_labels))

Epoch 1/5
215/215 [==============================] - 5s 19ms/step - loss: 0.2323 - accuracy: 0.9057 - val_loss: 0.5319 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1598 - accuracy: 0.9423 - val_loss: 0.5956 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1281 - accuracy: 0.9534 - val_loss: 0.7568 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1092 - accuracy: 0.9606 - val_loss: 0.7570 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.0920 - accuracy: 0.9660 - val_loss: 0.7342 - val_accuracy: 0.7874


In [38]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 1ms/step


array([[6.2093776e-02],
       [8.3167213e-01],
       [9.9965918e-01],
       [3.9496008e-02],
       [8.4988755e-04],
       [9.8857903e-01],
       [6.5730685e-01],
       [9.9978799e-01],
       [9.9955243e-01],
       [6.7678499e-01]], dtype=float32)

In [39]:
# Convert model 3 pred_probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [40]:
model_3_results = calculate_results(val_labels,model_3_preds)
model_3_results

{'accuracy': 78.74015748031496,
 'precision': 0.7894561193095897,
 'recall': 0.7874015748031497,
 'f1': 0.7854735566961013}

#### Model4 - Bidirectional RNN

In [41]:
# Trying out on my own
text_vectorizer_exp = tf.keras.layers.TextVectorization(max_tokens=10000,
                                      output_sequence_length=15)
text_vectorizer_exp.adapt(train_sentences)

inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer_exp(inputs)
x = tf.keras.layers.Embedding(input_dim=10000,
                              output_dim=128,
                              input_length=15)(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_4_exp = tf.keras.Model(inputs,outputs)
model_4_exp.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])
model_4_history_exp = model_4_exp.fit(train_sentences,
                                      train_labels,
                                      validation_data=(val_sentences,val_labels),
                                      epochs=5)

Epoch 1/5
215/215 [==============================] - 8s 23ms/step - loss: 0.5055 - accuracy: 0.7520 - val_loss: 0.4536 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 2s 7ms/step - loss: 0.3130 - accuracy: 0.8740 - val_loss: 0.5152 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2088 - accuracy: 0.9216 - val_loss: 0.5610 - val_accuracy: 0.7572
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1425 - accuracy: 0.9540 - val_loss: 0.6852 - val_accuracy: 0.7612
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0996 - accuracy: 0.9648 - val_loss: 0.7200 - val_accuracy: 0.7625


Normal RNN's go from left to right

Bi-directional go from right to left as well as left to right

In [42]:
# model 4
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=10000,
                                                    output_sequence_length=15)
text_vectorizer.adapt(train_sentences)

inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = tf.keras.layers.Embedding(input_dim=10000,
                              output_dim=128,
                              input_length=15)(x)
# x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_4 = tf.keras.Model(inputs,outputs)

model_4.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_4_history = model_4.fit(train_sentences,train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels))

Epoch 1/5
215/215 [==============================] - 6s 20ms/step - loss: 0.5088 - accuracy: 0.7454 - val_loss: 0.4581 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3113 - accuracy: 0.8714 - val_loss: 0.5272 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2092 - accuracy: 0.9204 - val_loss: 0.5773 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1428 - accuracy: 0.9520 - val_loss: 0.6659 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1006 - accuracy: 0.9660 - val_loss: 0.6933 - val_accuracy: 0.7677


In [43]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 0s 1ms/step


array([[0.02518224],
       [0.8288253 ],
       [0.9994019 ],
       [0.14624396],
       [0.00539156],
       [0.99798965],
       [0.88434756],
       [0.9995727 ],
       [0.99956816],
       [0.21081477]], dtype=float32)

In [44]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [45]:
model_4_results = calculate_results(val_labels,model_4_preds)
model_4_results

{'accuracy': 76.77165354330708,
 'precision': 0.7679905783589133,
 'recall': 0.7677165354330708,
 'f1': 0.7663871505080737}

#### 1D CNN

In [46]:
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = tf.keras.layers.Conv1D(32,
                                 5,
                                 activation="relu",
                                 padding="valid")
conv_1d_output = conv_1d(embedding_test)

max_pool = tf.keras.layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [47]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02745367,  0.04983198, -0.02139239, ...,  0.03518584,
          0.01700259, -0.0483982 ],
        [-0.05949971,  0.04479698, -0.02952983, ..., -0.06937435,
          0.02570894,  0.02816616],
        [-0.05402016, -0.01759368, -0.02602993, ...,  0.03084477,
         -0.02211493,  0.02448285],
        ...,
        [ 0.01937979,  0.01162951, -0.00827661, ..., -0.00865137,
          0.0256858 , -0.02262224],
        [ 0.01937979,  0.01162951, -0.00827661, ..., -0.00865137,
          0.0256858 , -0.02262224],
        [ 0.01937979,  0.01162951, -0.00827661, ..., -0.00865137,
          0.0256858 , -0.02262224]]], dtype=float32)>

In [48]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[0.10896946, 0.        , 0.02907739, 0.05390075, 0.02455719,
         0.        , 0.01336014, 0.04423492, 0.03102152, 0.        ,
         0.0317586 , 0.        , 0.        , 0.02333548, 0.        ,
         0.00820252, 0.        , 0.00179943, 0.06839722, 0.        ,
         0.        , 0.04786102, 0.00911995, 0.        , 0.04931985,
         0.        , 0.        , 0.01115392, 0.04931238, 0.        ,
         0.        , 0.04750063],
        [0.05020577, 0.        , 0.04047318, 0.01633164, 0.02994521,
         0.01250608, 0.04935616, 0.        , 0.        , 0.        ,
         0.02683765, 0.0067036 , 0.        , 0.        , 0.0643282 ,
         0.        , 0.        , 0.04386348, 0.01552715, 0.        ,
         0.        , 0.        , 0.05625538, 0.        , 0.04695395,
         0.02429056, 0.07400136, 0.        , 0.01576889, 0.04868954,
         0.        , 0.        ],
        [0.        , 0.        , 0.01151161, 0.0187

In [49]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.10896946, 0.        , 0.04047318, 0.05390075, 0.06528088,
        0.04103297, 0.04935616, 0.0462786 , 0.03102152, 0.00561017,
        0.03792642, 0.0534593 , 0.        , 0.02590381, 0.0643282 ,
        0.02232456, 0.        , 0.05033882, 0.06839722, 0.06301952,
        0.00070885, 0.04786102, 0.09360813, 0.00997178, 0.04931985,
        0.02429056, 0.0818077 , 0.02901478, 0.05637617, 0.06251238,
        0.01470468, 0.04750063]], dtype=float32)>

In [50]:
# Create 1D CNN layer to model sequences
inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.Conv1D(filters=64,
                           kernel_size=5,
                           padding="valid",
                           activation="relu")(x)
x = tf.keras.layers.GlobalMaxPool1D()(x)
outputs = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_5 = tf.keras.Model(inputs,outputs)

model_5.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_5_history = model_5.fit(train_sentences,train_labels,
                              validation_data=(val_sentences,val_labels),
                              epochs=5)

Epoch 1/5


215/215 [==============================] - 5s 20ms/step - loss: 0.1523 - accuracy: 0.9480 - val_loss: 0.7438 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1025 - accuracy: 0.9618 - val_loss: 0.8880 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0807 - accuracy: 0.9683 - val_loss: 0.9547 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.0693 - accuracy: 0.9734 - val_loss: 1.0408 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.0602 - accuracy: 0.9752 - val_loss: 1.1129 - val_accuracy: 0.7717


In [51]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 1ms/step


array([[2.4610125e-01],
       [5.9993929e-01],
       [9.9996984e-01],
       [4.9765490e-02],
       [2.8639894e-07],
       [9.9715459e-01],
       [9.8548234e-01],
       [9.9981397e-01],
       [9.9999881e-01],
       [7.5610954e-01]], dtype=float32)

In [52]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [53]:
model_5_results = calculate_results(val_labels,model_5_preds)
model_5_results

{'accuracy': 77.16535433070865,
 'precision': 0.7722289521502119,
 'recall': 0.7716535433070866,
 'f1': 0.7701831305177762}

In [54]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

#### Model 6 - Tensorflow Hub pretrained models
USE-extractor - universal sentence encoder (USE)

In [55]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_sample = embed([sample_sentence,
                      "When you can the unvidersal sentence encider on a sentence, it turns into into numbers"])
print(embed_sample[0][:50])

tf.Tensor(
[-0.01157027  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680985  0.05589838 -0.01068729 -0.00597292  0.00639323 -0.0181952
  0.00030814  0.09105888  0.05874645 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865346 -0.04209305  0.0267898   0.03011008  0.00321069
 -0.00337971 -0.04787356  0.02266719 -0.00985925 -0.04063613 -0.01292093
 -0.04666384  0.056303   -0.03949255  0.00517688  0.02495828 -0.07014441
  0.02871508  0.04947684 -0.00633978 -0.08960193  0.02807117 -0.00808362
 -0.01360601  0.0599865  -0.10361787 -0.05195374  0.00232955 -0.0233253
 -0.03758105  0.03327729], shape=(50,), dtype=float32)


In [56]:
embed_sample

<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[-0.01157027,  0.0248591 ,  0.02878048, ..., -0.00186125,
         0.02315824, -0.01485021],
       [ 0.06090042, -0.08857404,  0.01838204, ..., -0.00653699,
         0.03106262,  0.03701009]], dtype=float32)>

Use the above USE"" embedding layer for a normal type of model

In [57]:
# Create a keras layer using the USE"" pretrained layer from tensorflow hub
sentence_encoder_layer = hub.KerasLayer(hub.load("https://tfhub.dev/google/universal-sentence-encoder/4"),
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [58]:
sentence_encoder_layer

In [59]:
# Create  sequential layer
model_6 = tf.keras.models.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(1,activation="sigmoid")
])

model_6.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_6_history = model_6.fit(train_sentences,train_labels,
                               validation_data=(val_sentences,val_labels),
                               epochs=5)

Epoch 1/5
215/215 [==============================] - 3s 7ms/step - loss: 0.6482 - accuracy: 0.7378 - val_loss: 0.6154 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.5809 - accuracy: 0.7913 - val_loss: 0.5653 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.5378 - accuracy: 0.7930 - val_loss: 0.5333 - val_accuracy: 0.7822
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.5091 - accuracy: 0.7973 - val_loss: 0.5119 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4891 - accuracy: 0.7999 - val_loss: 0.4973 - val_accuracy: 0.7887


In [60]:
model_6_pred_probs=model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 0s 4ms/step


array([[0.37072343],
       [0.67399824],
       [0.849519  ],
       [0.3633444 ],
       [0.6382592 ],
       [0.7322892 ],
       [0.8246749 ],
       [0.84339666],
       [0.7467693 ],
       [0.1936599 ]], dtype=float32)

In [61]:
model_6_preds=tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [62]:
model_6_results = calculate_results(val_labels,model_6_preds)
model_6_results

{'accuracy': 78.87139107611549,
 'precision': 0.7896925793067333,
 'recall': 0.7887139107611548,
 'f1': 0.7873013461184764}

In [63]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [64]:
# Challenge, beat the baseline
model_6_exp = tf.keras.models.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(64,activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(64,activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)
])
model_6_exp.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])
model_6_exp_history = model_6_exp.fit(train_sentences,train_labels,
                                      epochs=5,
                                      validation_data=(val_sentences,val_labels))

model_6_exp_pred_probs = model_6_exp.predict(val_sentences)
print(model_6_exp_pred_probs[:10])

model_6_exp_preds = tf.squeeze(tf.round(model_6_exp_pred_probs))
print(model_6_exp_preds[:10])

model_6_exp_results = calculate_results(val_labels, model_6_exp_preds)
print(model_6_exp_results)

Epoch 1/5
215/215 [==============================] - 2s 7ms/step - loss: 0.4821 - accuracy: 0.7857 - val_loss: 0.4378 - val_accuracy: 0.8005
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.4011 - accuracy: 0.8222 - val_loss: 0.4234 - val_accuracy: 0.8163
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3788 - accuracy: 0.8345 - val_loss: 0.4219 - val_accuracy: 0.8163
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3581 - accuracy: 0.8483 - val_loss: 0.4210 - val_accuracy: 0.8202
Epoch 5/5
24/24 [==============================] - 0s 5ms/step
[[0.16142817]
 [0.79830945]
 [0.99721247]
 [0.16722518]
 [0.6336406 ]
 [0.86117154]
 [0.9954034 ]
 [0.9934882 ]
 [0.9897069 ]
 [0.06630458]]
tf.Tensor([0. 1. 1. 0. 1. 1. 1. 1. 1. 0.], shape=(10,), dtype=float32)
{'accuracy': 82.1522309711286, 'precision': 0.822740944702772, 'recall': 0.821522309711286, 'f1': 0.8204589074026987}


### Model 7 : TF Hub Pretrained USE but with 10% of training data

In [65]:
train_10_percent = train_df_shuffled[["text", "target"]].sample(frac=0.1, random_state=42)
train_sentences_10_percent = train_10_percent["text"].to_list()
train_labels_10_percent = train_10_percent["target"].to_list()
len(train_sentences_10_percent), len(train_labels_10_percent)

(761, 761)

In [66]:
# Check the number of targets in the dataset
train_10_percent["target"].value_counts()

target
0    413
1    348
Name: count, dtype: int64

In [67]:
train_df_shuffled["target"].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [68]:
# model 7
model_7 = tf.keras.models.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(64,activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(64,activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)
])

model_7.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_7_history = model_7.fit(train_sentences_10_percent, train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences,val_labels))


Epoch 1/5
24/24 [==============================] - 1s 22ms/step - loss: 0.6699 - accuracy: 0.6413 - val_loss: 0.6290 - val_accuracy: 0.7703
Epoch 2/5
24/24 [==============================] - 0s 11ms/step - loss: 0.5775 - accuracy: 0.8029 - val_loss: 0.5062 - val_accuracy: 0.8215
Epoch 3/5
24/24 [==============================] - 0s 11ms/step - loss: 0.4617 - accuracy: 0.8213 - val_loss: 0.4047 - val_accuracy: 0.8412
Epoch 4/5
24/24 [==============================] - 0s 11ms/step - loss: 0.3875 - accuracy: 0.8423 - val_loss: 0.3459 - val_accuracy: 0.8622
Epoch 5/5
24/24 [==============================] - 0s 11ms/step - loss: 0.3380 - accuracy: 0.8620 - val_loss: 0.3001 - val_accuracy: 0.8780


In [69]:
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

24/24 [==============================] - 0s 6ms/step


array([[0.09183506],
       [0.8766855 ],
       [0.955973  ],
       [0.22970389],
       [0.9057463 ],
       [0.8609012 ],
       [0.95389783],
       [0.97209686],
       [0.9284693 ],
       [0.01393676]], dtype=float32)

In [70]:
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [71]:
model_7_results = calculate_results(val_labels,model_7_preds)
model_7_results

{'accuracy': 87.79527559055119,
 'precision': 0.8787082788740477,
 'recall': 0.8779527559055118,
 'f1': 0.8775165403027484}

model 7 did very good compared to model 6, model is same, but only 10% data, why?
what could the reason be:
1. maybe the way we created 10% data

ANS : train_sentences_10_percent has little amount of val_sentences, becuase we took 10% from whole train_df_shuffled variable

So, it is already seeing what it will be validating on.

This is called data leak problem, some part of validation/testing data leaks into training data

In [72]:
# Making correct data allocation - without data leakage
train_10_percent_split = int(0.1 * len(train_sentences))    # train_sentences are already in random order, no need to shuffle
train_sentence_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]
len(train_sentence_10_percent), len(train_labels_10_percent)

(685, 685)

In [73]:
model_7 = tf.keras.models.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(64,activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(64,activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)
])
model_7.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
mdoel_7_history_split = model_7.fit(train_sentence_10_percent, train_labels_10_percent,
                                    epochs=5,
                                    validation_data=(val_sentences,val_labels))

Epoch 1/5
22/22 [==============================] - 3s 108ms/step - loss: 0.6687 - accuracy: 0.6219 - val_loss: 0.6524 - val_accuracy: 0.5997
Epoch 2/5
22/22 [==============================] - 0s 12ms/step - loss: 0.5799 - accuracy: 0.7635 - val_loss: 0.5579 - val_accuracy: 0.7717
Epoch 3/5
22/22 [==============================] - 0s 13ms/step - loss: 0.4548 - accuracy: 0.8204 - val_loss: 0.4954 - val_accuracy: 0.7940
Epoch 4/5
22/22 [==============================] - 0s 13ms/step - loss: 0.3764 - accuracy: 0.8438 - val_loss: 0.5009 - val_accuracy: 0.7808
Epoch 5/5
22/22 [==============================] - 0s 14ms/step - loss: 0.3252 - accuracy: 0.8642 - val_loss: 0.5104 - val_accuracy: 0.7730


In [74]:
model_7_pred_probs = model_7.predict(val_sentences)
print(model_7_pred_probs[:10])

model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
print(model_7_preds[:10])

model_7_results = calculate_results(val_labels, model_7_preds)
print(model_7_results)

24/24 [==============================] - 0s 5ms/step
[[0.02782756]
 [0.47674778]
 [0.9636344 ]
 [0.4345637 ]
 [0.64019966]
 [0.678855  ]
 [0.9527489 ]
 [0.8586304 ]
 [0.9380611 ]
 [0.09271669]]
tf.Tensor([0. 0. 1. 0. 1. 1. 1. 1. 1. 0.], shape=(10,), dtype=float32)
{'accuracy': 77.29658792650919, 'precision': 0.7738445106757977, 'recall': 0.7729658792650919, 'f1': 0.7713337273803944}


### Compare the performance of each of our model

In [ ]:
all_model_results = pd.DataFrame({"baseline":baseline_results,
                                  "simple_dense":model_1_results,
                                  "lstm":model_2_results})